In [164]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
import plotly.express as px
import plotly.graph_objects as go

In [165]:
# 경고메세지 끄기
import warnings
warnings.filterwarnings("ignore")

In [188]:
# 데이터베이스 table 가져오기
base_path = os.getcwd()
click_stream_new = pd.read_csv(base_path + '/click_stream_final.csv')

In [189]:
click_stream_new['event_time'] = pd.to_datetime(click_stream_new['event_time'])

In [190]:
click_stream_new['month'] = click_stream_new['event_time'].dt.strftime('%Y-%m')

In [191]:
# 세션 재정의 (HOMEPAGE 접속 기준)
new_session_idx = list(click_stream_new[click_stream_new['event_name'] == 'HOMEPAGE'].index)
new_session_idx.append(len(click_stream_new))

# HOMEPAGE에 접속한 것을 기준으로 session을 분할
new_session = []

# 스택을 활용하여 뒤에서부터 세션 분할
for i in range(len(new_session_idx) - 2, -1, -1):
    for j in range(new_session_idx[-1] - new_session_idx[-2]):
        new_session.append(i)
    new_session_idx.pop()

# 뒤에서부터 분할했던 세션을 다시 원래 순서로 돌려서 적용
click_stream_new['session_id'] = new_session[::-1]

### 퍼널 분석

퍼널 (HOMEPAGE -> ITEM_DETAIL -> ADD_TO_CART -> BOOKING)

In [192]:
# 행동분석을 위하여 session에 벌어진 모든 이벤트들이 첫 접속시점을 기준으로 해당 날짜에 모두 일어났다고 가정
session_time_mapping = click_stream_new.groupby('session_id').head(1)[['session_id', 'event_time']].set_index('session_id')['event_time'].to_dict()
click_stream_new['event_time'] = click_stream_new['session_id'].map(session_time_mapping)
click_stream_new['event_time'] = click_stream_new['event_time'].dt.strftime('%Y-%m')

In [194]:
# 타임, 세션별 이벤트
time_id_event = click_stream_new.groupby(['event_time', 'session_id'])['event_name'].unique()

In [227]:
# 퍼널 간 전환율 통합 데이터프레임

# HOMEPAGE -> ITEM_DETAIL 퍼널 전환율
# 접속한 사람 수 중 ITEM_DETAIL에 접속한 사람 수
home_to_item = pd.DataFrame(time_id_event.map(lambda x: 'ITEM_DETAIL' in x).groupby('event_time').sum() / time_id_event.map(lambda x: 'HOMEPAGE' in x).groupby('event_time').sum()).reset_index()

# ITEM_DETAIL -> ADD_TO_CART 퍼널 전환율
# ITEM_DETAIL에 접속한 사람 중 ADD_TO_CART를 한 사람 수
item_to_cart = pd.DataFrame(time_id_event.map(lambda x: ('ITEM_DETAIL' in x) and ('ADD_TO_CART' in x)).groupby('event_time').sum() / time_id_event.map(lambda x: 'ITEM_DETAIL' in x).groupby('event_time').sum()).reset_index()

# ADD_TO_CART -> BOOKING 퍼널 전환율
# ADD_TO_CART를 한 사람 중 BOOKING까지 이어진 사람 수
cart_to_book = pd.DataFrame(time_id_event.map(lambda x: ('ITEM_DETAIL' in x) and ('ADD_TO_CART' in x) and ('BOOKING' in x)).groupby('event_time').sum() / time_id_event.map(lambda x: ('ITEM_DETAIL' in x) and ('ADD_TO_CART' in x)).groupby('event_time').sum())

# 데이터프레임 통합
funnel_cvr = pd.merge(home_to_item, item_to_cart, how = 'inner', on = 'event_time')
funnel_cvr = pd.merge(funnel_cvr, cart_to_book, how = 'inner', on = 'event_time')

funnel_cvr.rename({'event_name_x' : 'home_to_item', 'event_name_y' : 'item_to_cart', 'event_name' : 'cart_to_book'}, axis = 1, inplace = True)

In [220]:
# 월별 퍼널간 전환율 변화 그래프 그리기
def funnel_cvr_graph(funnel, title, range, vline_pos):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = funnel_cvr['event_time'], y=round(funnel_cvr[funnel] * 100, 2), mode='lines+markers', name='Micro_CVR'))
    title = {'text':title, 'y':0.95,
            'x':0.5,'xanchor':'center', 'yanchor':'top', "font" : {"size":25}}
    fig.update_xaxes(title='월', title_font=dict(size=22))
    fig.update_yaxes(title='전환율 (%)', range=range, title_font=dict(size=22))
    fig.update_layout(title=title, width = 1000, height = 500)

    # 수직선을 그릴 위치
    vertical_line_position = '2022-06'

    fig.add_annotation(text='2022-06', x=vertical_line_position, y=vline_pos,
                    showarrow=False, font=dict(size=12, color='red'))
    # 수직선 추가
    fig.add_shape(type='line',
                x0=vertical_line_position, y0=0, x1=vertical_line_position, y1=1,
                line=dict(color='red', dash='dash'),
                xref='x', yref='paper')
    fig.show()

In [230]:
# 월별 홈페이지 → 상품 페이지 전환율 변화
funnel_cvr_graph('home_to_item', '월별 홈페이지 → 상품 페이지 전환율 변화', [20, 60], 60)

In [236]:
# 해당 구간의 전환율 증가는 분모인 홈페이지 방문 수가 더 크게 감소했기 때문

# 분자 감소율
print(f"분자(ITEM_DETAIL 접속 수) 감소율 : {time_id_event.map(lambda x: 'ITEM_DETAIL' in x).groupby('event_time').sum().pct_change().iloc[-1]}")

# 분모 감소율
print(f"분모(HOMEPAGE 접속 수) 감소율 : {time_id_event.map(lambda x: 'HOMEPAGE' in x).groupby('event_time').sum().pct_change().iloc[-1]}")

분자(ITEM_DETAIL 접속 수) 감소율 : -0.0674856894185385
분모(HOMEPAGE 접속 수) 감소율 : -0.1065718330123222


In [224]:
# 월별 상품 페이지 → 장바구니 전환율 변화
funnel_cvr_graph('item_to_cart', '월별 상품 페이지 → 장바구니 전환율 변화', [20, 60], 60)

In [237]:
# 분자 감소율
print(f"분자(ADD_TO_CART 접속 수) 감소율 : {time_id_event.map(lambda x: ('ITEM_DETAIL' in x) and ('ADD_TO_CART' in x)).groupby('event_time').sum().pct_change().iloc[-1]}")

# 분모 감소율
print(f"분모(ITEM_DETAIL 접속 수) 감소율 : {time_id_event.map(lambda x: 'ITEM_DETAIL' in x).groupby('event_time').sum().pct_change().iloc[-1]}")

분자(ITEM_DETAIL 접속 수) 감소율 : -0.2536627860932922
분모(HOMEPAGE 접속 수) 감소율 : -0.0674856894185385


In [228]:
# 월별 장바구니 → 결제 전환율 변화
funnel_cvr_graph('cart_to_book', '월별 장바구니 → 결제 전환율 변화', [20, 60], 60)

In [239]:
# 분자 감소율
print(f"분자(BOOKING 수) 감소율 : {time_id_event.map(lambda x: ('ITEM_DETAIL' in x) and ('ADD_TO_CART' in x) and ('BOOKING' in x)).groupby('event_time').sum().pct_change().iloc[-1]}")

# 분모 감소율
print(f"분모(ADD_TO_CART 접속 수) 감소율 : {time_id_event.map(lambda x: ('ITEM_DETAIL' in x) and ('ADD_TO_CART' in x)).groupby('event_time').sum().pct_change().iloc[-1]}")

분자(BOOKING 수) 감소율 : -0.3278958642280442
분모(ADD_TO_CART 접속 수) 감소율 : -0.2536627860932922


In [196]:
# ITEM_DETAIL 페이지 접속 인덱스
item_idx = list(click_stream_new[click_stream_new['event_name'] == 'ITEM_DETAIL'].index)

# 월별로 ADD_TO_CART 퍼널 전환 수 체크
cart_after_item = {k : v for k, v in zip(list(click_stream_new['event_time'].unique()), [0] * 24)}

# ITEM_DETAIL 페이지에 접속했을 때
for idx in item_idx:

    # 3번 이내에
    for i in range(1, 4):
        check = click_stream_new.loc[idx + i]
        # 다른 페이지로 이동하면 다음 session 체크
        if check['event_name'] in ['HOMEPAGE', 'SEARCH', 'ITEM_DETAIL', 'PROMO_PAGE', 'BOOKING', 'ADD_PROMO']:
            break

        # 장바구니에 담았다면 횟수 1 추가
        elif check['event_name'] == 'ADD_TO_CART':
            cart_after_item[check['event_time']] += 1
            break

cart_after_item = pd.DataFrame(cart_after_item.values(), cart_after_item.keys()).reset_index()
cart_after_item.columns = ['event_time', 'cart_cnt']
cart_after_item.sort_values(by='event_time', inplace=True)

# 전환율
cart_after_item.groupby('event_time')['cart_cnt'].sum() / click_stream_new[click_stream_new['event_name'] == 'ITEM_DETAIL'].groupby('event_time')['session_id'].size()

event_time
2020-08    0.164614
2020-09    0.157943
2020-10    0.159537
2020-11    0.161272
2020-12    0.156937
2021-01    0.154441
2021-02    0.159787
2021-03    0.155480
2021-04    0.155411
2021-05    0.154334
2021-06    0.149731
2021-07    0.153503
2021-08    0.143020
2021-09    0.146571
2021-10    0.140579
2021-11    0.141148
2021-12    0.136156
2022-01    0.137845
2022-02    0.133982
2022-03    0.128267
2022-04    0.121416
2022-05    0.115783
2022-06    0.097702
2022-07    0.057959
dtype: float64